# Factset

Notebook that extracts data from FastFech On-Demand Web Service.

In [ ]:
from datetime import datetime
import requests
import io
import json
import pandas as pd
from pathlib import Path

---
## 0. Configuration

Helper function to load JSON configuration files.

In [ ]:
def read_json(path):
    with open(path, 'r') as file:
        return json.load(file)

Retrieve configuration from files.

In [ ]:
CREDENTIALS = read_json('/Users/alex/.factset_credentials')

BALANCE_SHEET_CALL = read_json('calls/balance_sheet.json')
CASHFLOW_STATEMENT_CALL = read_json('calls/cashflow_statement.json')
ESG_CALL = read_json('calls/esg.json')
INCOME_STATEMENT_CALL = read_json('calls/income_statement.json')
LIST_CALL = read_json('calls/list.json')
PRICES_CALL = read_json('calls/prices.json')
RATIOS_CALL = read_json('calls/ratios.json')

OUTPUT_PATH = Path('./output/')

---
## 1. Call webservice

`_call_fastfetch()` defines the a skeleton call to a webservice.  
**Supported arguments:** currency, cutoff, date, dates, format, ids, ison,
isonParams, items, orientation, showDataType, showTitle, showHeader, universe, universeGroup.

Add username and password.

In [ ]:
def _call_fastfetch(**kwargs):

    """ base call for on-demand webservice """

    endpoint = kwargs.pop('endpoint')
    username = kwargs.pop('username')
    password = kwargs.pop('password')
    
    kwargs['format'] = 'csv'
    kwargs['showTitle'] = kwargs['showHeader'] = kwargs['showDatatype'] = 'N'

    items = kwargs.pop('items')
    # Generate comma-separated strings of keys and values, maintaining the same order
    column_list = list(items.keys())
    item_list = [items[key] for key in column_list]

    kwargs['items']=','.join(item_list)
    
    # Query the data from factset
    response = requests.post(endpoint,
                             auth = (username, password),
                             data = kwargs)

    #print(response.text)
    
    # If the status code is not success (200) raise exception
    if not response.status_code == 200:
        raise Exception(f'Error in request. Response with status code: {response.status_code}')
    
    # transform the response variable to DataFrame
    data = pd.read_csv(io.StringIO(response.text))
    data.columns = ['id', 'date'] + column_list

    return data

---
## 2. Screening

`factset_extract_snapshot()` selects certain attributes `items` for desired assets.  
These assets are filtered based on conditions provided in `universe` in Factset Screening Language.

In [ ]:
def factset_extract_snapshot(universe, **kwargs):
    return _call_fastfetch(
            factlet='ExtractDataSnapshot',
            date='0',
            universe=universe,
            items=LIST_CALL,
            **kwargs)

---
## 3. Historical data

`factset_extract_formula_history()` returns a panel of symbol/date features.

In [ ]:
def factset_extract_formula_history(**kwargs):

    """ historical data data """

    ids = kwargs.pop('ids', None)

    if ids:
        # replace with a string
        ids = ','.join(ids)
        kwargs['ids']= ids
    
    items = kwargs.pop('items')
    
    # Generate comma-separated strings of keys and values, maintaining the same order
    column_list = list(items.keys())
    item_list = [ items[item].format(**kwargs) for item in column_list ]

    kwargs['items'] = { k:v for (k,v) in zip(column_list, item_list)}  
    kwargs['dates'] = '{t1}:{t2}:{frequency}'.format(**kwargs)
    
    data = _call_fastfetch(factlet='ExtractFormulaHistory', **kwargs)

    # parse date
    data['date'] = pd.to_datetime(data['date'])

    return data

---
## 4. Utility functions

In [ ]:
def screening(universes):

    results = []

    for universe in UNIVERSES:
    
        print(f'factset_retrieve_listing\n\t{universe=}')
        
        df = factset_extract_snapshot(universe=universe, **CREDENTIALS)
        df['universe'] = universe
        results.append(df)
        
        retrieved = len(df)
        print(f'\tdone, {retrieved} rows')
        
    df_listing = pd.concat(results)
    
    len_before = len(df_listing)
    df_listing = df_listing.drop_duplicates(subset=['id'])
    df_listing = df_listing.set_index('id')
    len_after = len(df_listing)
    
    print(f'removed duplicates: {len_before=} {len_after=}')

    return df_listing

---
## 5. EXTRACTION

First, filter the assets to download.

In [ ]:
# SP50
# IBEX-MCE
# 990100
# Energy assets (25), share, NYSE
# universe="(PUBLIC:FR_RBICS_NUM_CURR='25' AND PUBLIC:FREF_SECURITY_TYPE='SHARE' AND P_EXCHANGE='NYSE')=1"
#"(PUBLIC:F_COUNTRY='SPAIN' AND PUBLIC:FS_SEC_TYPE='SHARE' AND PUBLIC:ISON_ES_INDEX(10,IGBM))=1"
#universe = f'FG_CONSTITUENTS({source},0,CLOSE)'

SOURCES = ['SP50', 'IBEX-MCE']
SOURCES = ['SP50']

UNIVERSES = [
    #"(FR_RBICS_NUM_CURR='25' AND FREF_SECURITY_TYPE='SHARE' AND P_EXCHANGE='NYSE')=1",
    "FG_CONSTITUENTS(SP50,0,CLOSE)"
]

df_listing = screening(UNIVERSES)
df_listing.to_csv(OUTPUT_PATH / 'listing.csv.gz')
df_listing.to_excel(OUTPUT_PATH / 'listing.xlsx')

In [ ]:
# Extract daily prices
df_prices = factset_extract_formula_history(
    ids=df_listing.index.tolist(),
    t1='0', t2='20000101', frequency='D', currency='USD',
    items=PRICES_CALL, **CREDENTIALS)

df_prices.to_csv(OUTPUT_PATH / 'daily_prices.csv.gz')

if len(df) < 1048576:
    df_prices.to_excel(OUTPUT_PATH / 'daily_prices.xlsx')
else:
    print('Cannot save Excel file: Max sheet size is 1048576 rows x 16384 columns')

df_prices.tail()

In [ ]:
# Extract monthly prices
df_prices = factset_extract_formula_history(
    ids=df_listing.index.tolist(),
    t1='0', t2='20000101', frequency='M', currency='USD',
    items=PRICES_CALL, **CREDENTIALS)

df_prices.to_csv(OUTPUT_PATH / 'monthly_prices.csv.gz')
df_prices.to_excel(OUTPUT_PATH / 'monthly_prices.xlsx')

df_prices.tail()

In [ ]:
# Extract ESG
df_esg = factset_extract_formula_history(
    ids=df_listing.index.tolist(),
    t1='0', t2='20230101', frequency='M', currency='USD', 
    items=ESG_CALL, **CREDENTIALS)

df_esg.to_csv(OUTPUT_PATH / 'esg.csv.gz')
df_esg.to_excel(OUTPUT_PATH / 'esg.xlsx')

df_esg.tail()

In [ ]:
# Extract balance sheet
df_bs = factset_extract_formula_history(
    ids=df_listing.index.tolist(),
    t1='0', t2='20000101', frequency='Q', basis='QTR_R', currency='USD', 
    items=BALANCE_SHEET_CALL, **CREDENTIALS)

df_bs.to_csv(OUTPUT_PATH / 'balance_sheet.csv.gz')
df_bs.to_excel(OUTPUT_PATH / 'balance_sheet.xlsx')

df_bs.tail()

In [ ]:
# Extract income statement
df_is = factset_extract_formula_history(
    ids=df_listing.index.tolist(),
    t1='0', t2='20000101', frequency='Q', basis='QTR_R', currency='USD', 
    items=INCOME_STATEMENT_CALL, **CREDENTIALS)

df_is.to_csv(OUTPUT_PATH / 'income_statement.csv.gz')
df_is.to_excel(OUTPUT_PATH / 'income_statement.xlsx')

df_is.tail()

In [ ]:
# Extract cashflow statement
df_cs = factset_extract_formula_history(
    ids=df_listing.index.tolist(),
    t1='0', t2='20000101', frequency='Q', basis='QTR_R', currency='USD', 
    items=CASHFLOW_STATEMENT_CALL, **CREDENTIALS)

df_cs.to_csv(OUTPUT_PATH / 'cashflow_statement.csv.gz')
df_cs.to_excel(OUTPUT_PATH / 'cashflow_statement.xlsx')

df_cs.tail()

In [ ]:
# Extract ratios
RATIOS_CALL = read_json('calls/ratios.json')
df_r = factset_extract_formula_history(
    ids=df_listing.index.tolist(),
    t1='0', t2='20000101', frequency='Q', basis='QTR_R', currency='USD', 
    items=RATIOS_CALL, **CREDENTIALS)

df_r.to_csv(OUTPUT_PATH / 'ratios.csv.gz')
df_r.to_excel(OUTPUT_PATH / 'ratios.xlsx')

df_r.tail()